In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_6332/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_6332/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_47_7,0.894446,0.811201,0.614688,0.821723,0.038509,0.112079,0.087040,0.036674,0.100042,0.196238,0.879109,0.197708,40.513707,89.903379,"Hidden Size=[4], regularizer=0.02, learning_ra..."
1,model_1_48_2,0.895067,0.808794,0.601579,0.813619,0.038283,0.113508,0.090001,0.044172,0.098984,0.195660,0.879820,0.197126,40.525499,89.915170,"Hidden Size=[4], regularizer=0.02, learning_ra..."
2,model_1_49_7,0.894451,0.807478,0.608664,0.908236,0.038507,0.114290,0.088400,0.030877,0.097876,0.196233,0.879115,0.197703,40.513806,89.903477,"Hidden Size=[4], regularizer=0.02, learning_ra..."
3,model_2_4_3,0.936693,0.806694,0.747855,0.932284,0.023096,0.114755,0.056958,0.030275,0.077356,0.151974,0.916009,0.153113,73.536172,169.410239,"Hidden Size=[8], regularizer=0.02, learning_ra..."
4,model_1_49_3,0.894514,0.806606,0.603041,0.910965,0.038485,0.114807,0.089671,0.029959,0.097878,0.196175,0.879187,0.197644,40.514995,89.904667,"Hidden Size=[4], regularizer=0.02, learning_ra..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
856,model_4_0_4,-0.032019,-0.095604,-1.956516,0.144177,0.376513,0.650398,0.667859,0.504114,0.352296,0.613606,-2.737583,0.618203,195.953608,477.765261,"Hidden Size=[24], regularizer=0.02, learning_r..."
857,model_5_18_0,0.905770,-0.098240,-0.670254,0.889040,0.034378,0.651963,0.377300,0.025669,0.074938,0.185413,2.147896,0.186802,296.740669,718.005512,"Hidden Size=[36], regularizer=0.02, learning_r..."
858,model_5_17_9,0.904041,-0.099008,-0.667564,0.696039,0.035009,0.652418,0.376693,0.121904,0.075534,0.187107,2.168957,0.188508,296.704308,717.969150,"Hidden Size=[36], regularizer=0.02, learning_r..."
859,model_4_0_0,-0.282966,-0.229298,-2.553724,-0.046401,0.468066,0.729764,0.802765,0.616373,0.408095,0.684153,-3.646419,0.689278,195.518292,477.329946,"Hidden Size=[24], regularizer=0.02, learning_r..."
